In [2]:
import pandas as pd
from flask import Flask, render_template, request, redirect, url_for, session, send_file
from io import StringIO, BytesIO
from threading import Thread
from datetime import datetime, timedelta

app = Flask(__name__)
app.secret_key = 'sua_chave_secreta'  # Use uma chave segura para sessões

# Função para carregar dados e filtrar (já existentes)
def carregar_dados_por_data(data, jogou, joogou_ante):
    print(f"[DEBUG] Carregando dados para data: {data}, Jogou: {jogou}, Joogou Ante: {joogou_ante}")
    
    with open('DadosApostaganha.txt', 'r') as file:
        linhas = file.readlines()

    colunas = linhas[0].strip().split(';')
    dados = [linha.strip().split(';') for linha in linhas[1:]]
    
    df = pd.DataFrame(dados, columns=colunas)
    df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y', dayfirst=True, errors='coerce')

    data_selecionada = pd.to_datetime(data, format='%Y-%m-%d', errors='coerce')
    df_filtrado = df[df['Data'] == data_selecionada].copy()  # Garante que seja uma cópia

    if jogou:
        df_filtrado = df_filtrado[df_filtrado['Jogou'] == jogou].copy()

    if joogou_ante:
        df_filtrado = df_filtrado[df_filtrado['joogou_ante'] == joogou_ante].copy()

    df_filtrado['Valor Primeira Linha'] = pd.to_numeric(df_filtrado['Valor Primeira Linha'], errors='coerce')

    print(f"[DEBUG] Dados filtrados encontrados: {len(df_filtrado)} linhas")
    
    return df_filtrado

# Função para verificar o usuário
def verificar_usuario(nome_usuario, senha):
    with open('usuarios.txt', 'r') as file:
        for linha in file:
            usuario, senha_armazenada = linha.strip().split(';')
            if usuario == nome_usuario and senha_armazenada == senha:
                return True
    return False

# Rota para download do CSV
@app.route('/download_csv', methods=['POST'])
def download_csv():
    print("[DEBUG] Botão 'Exportar para CSV' foi clicado!")

    data_selecionada = request.form['data']
    jogou_selecionado = request.form.get('jogou', '')  
    jogou_ante_selecionado = request.form.get('joogou_ante', '')

    dados_filtrados = carregar_dados_por_data(data_selecionada, jogou_selecionado, jogou_ante_selecionado)

    if dados_filtrados.empty:
        print("[DEBUG] Nenhum dado encontrado para exportação.")
        return "Nenhum dado encontrado para exportação.", 400

    try:
        # Salva em um buffer de memória (usando BytesIO)
        output = BytesIO()
        dados_filtrados.to_csv(output, index=False, sep=";")
        output.seek(0)

        print("[DEBUG] Arquivo CSV gerado com sucesso. Enviando para download...")

        response = send_file(output, as_attachment=True, download_name="dados_filtrados.csv", mimetype="text/csv")

        print("[DEBUG] Arquivo enviado com sucesso!")

        return response
    except Exception as e:
        print(f"[ERRO] Falha ao gerar/enviar o CSV: {str(e)}")
        return "Erro ao gerar o arquivo CSV.", 500

# Rota para login e resultados
@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        nome_usuario = request.form['username']
        senha = request.form['password']
        if verificar_usuario(nome_usuario, senha):
            session['usuario'] = nome_usuario
            return redirect(url_for('index'))
        else:
            return render_template('login.html', error='Usuário ou senha incorretos.')

    return render_template('login.html')

@app.route('/resultados', methods=['GET', 'POST'])
def index():
    if 'usuario' not in session:
        return redirect(url_for('login'))  # Redireciona para login caso não esteja autenticado

    dados_filtrados = pd.DataFrame()
    if request.method == 'POST':
        print("[DEBUG] Formulário de resultados submetido")
        data_selecionada = request.form['data']
        jogou_selecionado = request.form.get('jogou', '')
        jogou_ante_selecionado = request.form.get('joogou_ante', '')
        dados_filtrados = carregar_dados_por_data(data_selecionada, jogou_selecionado, jogou_ante_selecionado)

    return render_template('resultados.html', df=dados_filtrados)

# Função duplicada para carregar dados (conforme código original)
def carregar_dados_por_data(data, jogou, joogou_ante):
    with open('DadosApostaganha.txt', 'r') as file:
        linhas = file.readlines()
    
    colunas = linhas[0].strip().split(';')
    dados = [linha.strip().split(';') for linha in linhas[1:]]
    
    df = pd.DataFrame(dados, columns=colunas)
    df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y', dayfirst=True, errors='coerce')
    df['Valor Primeira Linha'] = pd.to_numeric(df['Valor Primeira Linha'], errors='coerce')
    
    data_selecionada = pd.to_datetime(data, format='%Y-%m-%d', errors='coerce')
    df_filtrado = df[df['Data'] == data_selecionada].copy()
    
    if jogou:
        df_filtrado = df_filtrado[df_filtrado['Jogou'] == jogou].copy()
    if joogou_ante:
        df_filtrado = df_filtrado[df_filtrado['joogou_ante'] == joogou_ante].copy()
    
    return df_filtrado

def calcular_lucro(valor, aposta, valor_jogada):
    print(f"Calculando lucro para valor: {valor}, aposta: {aposta}, ValorJogada: {valor_jogada}")  # Debug
    if valor < valor_jogada:
        resultado = aposta * -1  # Perda
    else:
        resultado = (aposta * valor_jogada) - aposta  # Ganho
    print(f"Resultado: {resultado}")  # Imprime o resultado do cálculo
    return resultado

@app.route('/simulacao', methods=['GET', 'POST'])
def simulacao():
    if 'usuario' not in session:
        return redirect(url_for('login'))
    
    # Definir valores padrão
    data_atual = datetime.today().strftime('%Y-%m-%d')
    valor_aposta_padrao = 1.00
    valor_jogada_padrao = 2.98

    resultado = None
    if request.method == 'POST':
        data = request.form.get('data', data_atual)
        valor_aposta = float(request.form.get('valor_aposta', valor_aposta_padrao))
        valor_jogada = float(request.form.get('valor_jogada', valor_jogada_padrao))
        jogou = request.form.get('jogou', '')
        jogou_ante = request.form.get('joogou_ante', '')

        df_filtrado = carregar_dados_por_data(data, jogou, jogou_ante)
        
        if not df_filtrado.empty:
            df_filtrado = df_filtrado[df_filtrado['joogou_ante'] == 'jogado']
            df_filtrado['Lucro'] = df_filtrado['Valor Primeira Linha'].apply(
                lambda x: calcular_lucro(x, valor_aposta, valor_jogada)
            )

            total_green = (df_filtrado['Lucro'] > 0).sum()
            total_red = (df_filtrado['Lucro'] < 0).sum()
            lucro_green = df_filtrado.loc[df_filtrado['Lucro'] > 0, 'Lucro'].sum()
            lucro_red = df_filtrado.loc[df_filtrado['Lucro'] < 0, 'Lucro'].sum()
            lucro_total = df_filtrado['Lucro'].sum()
            porcentagem_green = (total_green / len(df_filtrado)) * 100 if len(df_filtrado) > 0 else 0

            resultado = {
                'total_green': total_green,
                'total_red': total_red,
                'porcentagem_green': round(porcentagem_green, 2),
                'lucro_total': f"R$ {lucro_total:,.2f}",
                'lucro_green': f"R$ {lucro_green:,.2f}",
                'lucro_red': f"R$ {lucro_red:,.2f}",
                'total_jogadas': len(df_filtrado)
            }

    return render_template(
        'simulacao.html',
        resultado=resultado,
        data_padrao=data_atual,
        valor_aposta_padrao=valor_aposta_padrao,
        valor_jogada_padrao=valor_jogada_padrao
    )

    
    return render_template('simulacao.html', resultado=resultado)

# Nova rota /velas_altas com filtro por data e hora
@app.route('/velas_altas', methods=['GET'])
def velas_altas():
    # Define os filtros padrão
    now = datetime.now()
    default_data_inicial = (now - timedelta(days=2)).date().isoformat()  # dois dias atrás
    default_data_final = now.date().isoformat()
    default_hora_inicial = "00:00:00"
    default_hora_final = "23:59:59"
    
    # Captura os parâmetros via query string (GET) ou usa os padrões
    data_inicial_str = request.args.get("data_inicial", default_data_inicial)
    data_final_str = request.args.get("data_final", default_data_final)
    hora_inicial_str = request.args.get("hora_inicial", default_hora_inicial)
    hora_final_str = request.args.get("hora_final", default_hora_final)
    
    # Se os campos de hora vierem no formato HH:MM (sem segundos), adiciona ":00"
    if len(hora_inicial_str) == 5:
        hora_inicial_str += ":00"
    if len(hora_final_str) == 5:
        hora_final_str += ":00"
    
    try:
        filtro_inicial = datetime.strptime(f"{data_inicial_str} {hora_inicial_str}", "%Y-%m-%d %H:%M:%S")
        filtro_final = datetime.strptime(f"{data_final_str} {hora_final_str}", "%Y-%m-%d %H:%M:%S")
    except Exception as e:
        filtro_inicial = datetime.strptime(f"{default_data_inicial} {default_hora_inicial}", "%Y-%m-%d %H:%M:%S")
        filtro_final = datetime.strptime(f"{default_data_final} {default_hora_final}", "%Y-%m-%d %H:%M:%S")
    
    dados = []
    try:
        with open('valores_salvos.txt', 'r') as file:
            linhas = file.readlines()
    except Exception as e:
        linhas = []
    
    for linha in linhas:
        linha = linha.strip()
        if linha:
            partes = linha.split(" - ")
            if len(partes) == 2:
                dt_str, valor_str = partes
                try:
                    dt_item = datetime.strptime(dt_str, "%Y-%m-%d %H:%M:%S")
                except Exception as e:
                    continue
                # Filtra os itens que estiverem dentro do intervalo definido
                if filtro_inicial <= dt_item <= filtro_final:
                    data_str = dt_item.date().isoformat()
                    hora_str = dt_item.strftime("%H:%M:%S")
                    try:
                        valor = float(valor_str)
                    except:
                        valor = valor_str
                    dados.append({'data': data_str, 'hora': hora_str, 'valor': valor, 'dt': dt_item})
    
    # Ordena os dados por data e hora de forma decrescente
    dados = sorted(dados, key=lambda x: x['dt'], reverse=True)
    # Remove o campo auxiliar 'dt'
    for item in dados:
        item.pop('dt', None)
    
    # Passa todas as variáveis para o template
    return render_template('velas_altas.html', 
                           dados=dados,
                           data_inicial=data_inicial_str,
                           data_final=data_final_str,
                           hora_inicial=hora_inicial_str,
                           hora_final=hora_final_str)


@app.route('/logout')
def logout():
    session.pop('usuario', None)  # Remove o usuário da sessão
    return redirect(url_for('login'))

def run_app():
    app.run(host='192.168.0.113', port=5035, debug=True, use_reloader=False)

if __name__ == '__main__':
    thread = Thread(target=run_app)
    thread.start()


 * Serving Flask app '__main__'
 * Debug mode: on


Cannot assign requested address
